In [1]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 40.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 28.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install xgboost 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 39.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 3.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
brew install libomp

SyntaxError: invalid syntax (2760538136.py, line 1)

In [6]:
pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.1 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for lightgbm (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [42 lines of output]
      2024-02-21 20:51:48,494 - scikit_build_core - INFO - RUN: /private/var/folders/b1/zqzn387j6zd37gnn4xj6ym200000gn/T/pip-build-env-nbvlvj02/normal/lib/python3.9/site-packages/cmake/data/bin/cmake --version
      2024-02-21 20:51:48,500 - scikit_build_core - INFO - CMake version: 3.28.3
      *** scikit-build-core 0.8.1 using CMake 3.28.3 (wheel)
      2024-02-21 20:51:48,502 - scikit_build_core - INFO - Build directory: /private/var/folders/b1/zqzn387j6zd37gnn4xj6ym200000gn/T/tmpu5o1qf37/build
      *** Configuring CMake...
      2024-02-21 20:51:48,511 - scikit_b

In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
#import optuna
from category_encoders import OneHotEncoder, MEstimateEncoder, CatBoostEncoder, OrdinalEncoder
from sklearn import set_config
import category_encoders
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, f1_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.preprocessing import FunctionTransformer,StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.metrics import auc, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier

ModuleNotFoundError: No module named 'xgboost'

In [6]:
# load data here
train = pd.read_csv('data/training.csv')
test = pd.read_csv('data/test.csv')


In [7]:
#Define cat and numeric columns
num_cols = train.select_dtypes('number').columns.tolist()
target = 'DiagPeriodL90D'
cat_cols = [c for c in train.columns if c not in num_cols and c != 'DiagPeriodL90D']
num_cols.remove('DiagPeriodL90D')


In [8]:
#make sure numeric cols doesn't have numeric categories
train[num_cols]

# apparently patient_id, patient_zip3 are cat cols
cat_cols.extend(['patient_id', 'patient_zip3'])
num_cols.remove('patient_id')
num_cols.remove('patient_zip3')

In [9]:
# drop column here, example of dropping  bmi
class DropColumns(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        x_copy = X.copy()        
        x_copy = x_copy.drop('bmi',axis=1) # drop column here
        return x_copy
    
# add column here, example of adding an all-one colum 
class AddColumns(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        x_copy = X.copy()        
        # x_copy['all_one'] = np.ones(len(x_copy)) # add column here
        return x_copy
    
# self-define missing value class   
class InputCol(TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        x_copy = X.copy()   
        return x_copy

NameError: name 'BaseEstimator' is not defined

In [56]:

def ImputeCatCols(d):
    df = d.copy()
    for c in cat_cols:
        df[c] = df[c].fillna('None')
    return df

num_transformer = Pipeline([('imputer',SimpleImputer(strategy='mean')), # set missing data to mean, other options are KNNImputer(n_neighbors=2)
                             ('scaler',StandardScaler())])
cat_transformer = Pipeline([('imputer',FunctionTransformer(ImputeCatCols)), 
                             ('cat',CatBoostEncoder())])

preprocess = ColumnTransformer([('num',num_transformer,num_cols),
                                ('cat',cat_transformer,cat_cols)],
                                remainder='passthrough',
                                verbose_feature_names_out=False).set_output(transform='pandas')

preprocess_catboost= Pipeline([('preprocess',preprocess),
                           ('drop',DropColumns())  # add or drop column here
                          ])

/Users/cs/Desktop/widsdatathon2024-challenge1/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


In [57]:
cat_transformer_2= Pipeline([('imputer',SimpleImputer(strategy='most_frequent')),
                           ('cat',LabelEncoder())])
num_transformer_2 = Pipeline([('imputer',SimpleImputer(strategy='most_frequent'))])
preprocess_3 = ColumnTransformer([('cat',cat_transformer_2,cat_cols),
                                            ('num',num_transformer_2,num_cols)
                                           ])

In [58]:
def score_model(model, label):
    
    print('model: {}'.format(label))
    X = train.copy()
    y = X.pop(target)
              
    skf = StratifiedKFold(n_splits=5,random_state=SEED, shuffle=True)
    
    val_predictions = np.zeros(len(train))
    score_list = []
    for fold, (trx_idx, val_idx) in enumerate(skf.split(X,y)):
        X_train = X.iloc[trx_idx]
        y_train = y.iloc[trx_idx]
        X_val   = X.iloc[val_idx]
        y_val   = y.iloc[val_idx]
        
        model.fit(X_train,y_train)
        y_pred_train = model.predict_proba(X_train)[:,1]
        y_pred_val   = model.predict_proba(X_val)[:,1]
        auc_train = roc_auc_score(y_train,y_pred_train)
        auc_val   = roc_auc_score(y_val,y_pred_val)
        
        val_predictions[val_idx] = y_pred_val
        score_list.append(auc_val)
        print(f'fold: {fold} - AUC Train: {auc_train} - AUC Val {auc_val}') 

    print(f'AUC MEAN {np.mean(score_list)} - Std: {np.std(score_list)}')  
    
    return score_list, val_predictions

In [59]:
SEED = 0


params_cat = {'learning_rate': 0.004, 
              'iterations': 1000,
              'max_depth': 5, 
              'subsample': 0.7401131867566202, 
              'colsample_bylevel': 0.29684187768021997, 
              'min_data_in_leaf': 47,
              'logging_level': 'Silent'}

params_lgb= {'learning_rate': 0.0016,
             'subsample': 0.6710494933148675, 
             'colsample_bytree': 0.7929648706646588, 
             'num_leaves': 29,
             'verbose':-1}


params_xbg = {'learning_rate': 0.001, 
              'max_depth': 6, 
              'subsample': 0.5281085467708261, 
              'min_child_weight': 9}

In [61]:
score_list, oof_list= pd.DataFrame(), pd.DataFrame()
models = [          
        #    ('catBoost',make_pipeline(preprocess_catboost,
        #                             CatBoostClassifier(cat_features=cat_cols,
        #                                                **params_cat,
        #                                                random_state=SEED))),                                       
                            
           ('RF',make_pipeline(preprocess_othermodels,
                               RandomForestClassifier(n_estimators=200,
                                                      random_state=SEED,
                                                      min_samples_leaf=92,
                                                      max_features=1.0))),
                                                      
            ('lgbm',make_pipeline(preprocess_othermodels,
                                 LGBMClassifier(**params_lgb,
                                                random_state=SEED))),  
                                                
           ('Extratrees',make_pipeline(preprocess_othermodels,
                                       ExtraTreesClassifier(n_estimators=300,
                                                           random_state=SEED,
                                                           min_samples_leaf=46,
                                                           max_features=1.0))),
           ('XGB',make_pipeline(preprocess_othermodels,
                                 XGBClassifier(**params_xbg,random_state=SEED)))                                        

            
        ]
for label, model in models:
    score_list[label], oof_list[label] = score_model(model,label)


model: RF
fold: 0 - AUC Train: 0.8502600865669502 - AUC Val 0.8010395615359034
fold: 1 - AUC Train: 0.8535540329460948 - AUC Val 0.7977654689928734
fold: 2 - AUC Train: 0.8528011997009787 - AUC Val 0.8078609986504722
fold: 3 - AUC Train: 0.8538171445651039 - AUC Val 0.7887176158173861
fold: 4 - AUC Train: 0.8595907395270574 - AUC Val 0.7779822128668629
AUC MEAN 0.7946731715726997 - Std: 0.010373704780898897
model: lgbm
fold: 0 - AUC Train: 0.835387853962925 - AUC Val 0.7981296845821287
fold: 1 - AUC Train: 0.8329491430341565 - AUC Val 0.7884986696781364
fold: 2 - AUC Train: 0.8297525018704827 - AUC Val 0.81016761543327
fold: 3 - AUC Train: 0.8410855823393162 - AUC Val 0.7862266233383781
fold: 4 - AUC Train: 0.8363818346218169 - AUC Val 0.780086528538541
AUC MEAN 0.7926218243140909 - Std: 0.010518782729787355
model: Extratrees
fold: 0 - AUC Train: 0.8797669399972344 - AUC Val 0.7986371543321993
fold: 1 - AUC Train: 0.8792950296685101 - AUC Val 0.8030723520961212
fold: 2 - AUC Train: 0.8

In [64]:
# voting from all models
w = RidgeClassifier().fit(oof_list,train.DiagPeriodL90D).coef_[0]
voter = VotingClassifier(models, weights = w, voting = 'soft')

X = train.copy()
y = X.pop('DiagPeriodL90D')      
                       
voter.fit(X,y)
voter.predict_proba(test)[:,1]


array([0.69988248, 0.66836308, 0.67904398, ..., 0.75151407, 0.2886226 ,
       0.71415358])